# IC Music Competition basic leaderboard Entry 
# Logistic classifier for 147 genres of music 

## Introduction

In this excecise I will begin by getting the summary vectors for each song and using them I will train logistic regression classifiers before using the classifiers to classify the different songs available to me.

[Kraggle page](https://www.kaggle.com/c/icmusicgenre)
[First leaderboard entry](http://course.dougturnbull.org/mediawiki/index.php/CS490_Project_1st_Entry)



In [85]:
%matplotlib inline  
import librosa
import librosa.display
import IPython
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import os 
from io import BytesIO
from sklearn.linear_model import LogisticRegression 
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.svm import SVC
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

## Summary song descriptor vector extraction
Iterate over each of the files within the song directory and get the summary features required for the learning algorithm then clean out any NaN instances by replacing them with 0

In [107]:

#89 with 14 + extra feature 
#75 with just feature summary 
dataMatrix = np.zeros((7868, 89), dtype=float)
testMatrix = np.zeros((2705, 89), dtype=float)
for i in range(7868):
    song_feature_directory_path = 'D:\\train_feature_files\\'
    song_feature_directory_path = song_feature_directory_path + str(i) + ".npz"
    data = np.load(song_feature_directory_path)
#     dataMatrix[i:] = data['summary']
    dataMatrix[i:,:76] = data['summary'] 
    dataMatrix[i:,76] = data["summary"][0] / (data["summary"][1] + 1)
    n = data['mfccs'].shape[1]
    f = np.mean(data['mfccs'][:,:int(n/2)], axis=1)
    s = np.mean(data['mfccs'][:,int(n/2):], axis=1)
    average = s/f
    dataMatrix[:,76:] = average
    
    
dataMatrix = np.nan_to_num(dataMatrix)

for i in range(2705):
    song_feature_directory_path = 'D:\\test_feature_files\\'
    song_feature_directory_path = song_feature_directory_path + str(i) + ".npz"
    data = np.load(song_feature_directory_path)
#     testMatrix[i:] = data['summary']
    testMatrix[i:,:76] = data['summary'] 
    testMatrix[i:,76] = data["summary"][0] / (data["summary"][1] + 1)
    n = data['mfccs'].shape[1]
    f = np.mean(data['mfccs'][:,:int(n/2)], axis=1)
    s = np.mean(data['mfccs'][:,int(n/2):], axis=1)
    average = s/f
    testMatrix[:,76:] = average

testMatrix =np.nan_to_num(testMatrix)

In [20]:

print(data['mfccs'].shape[1])
n = data['mfccs'].shape[1]

f = np.mean(data['mfccs'][:,:int(n/2)], axis=1)
s = np.mean(data['mfccs'][:,int(n/2):], axis=1)
average = s/f
print(average.shape)

6988
(13,)


## Setting up the Y labels

Load the cal10k_train_date matrix Y_train for the 147 genres and 7868 training tracks

In [52]:
# encode each of the string feature names to ints 
y_all = np.loadtxt(open("D:\\cal10k_train_data\\cal10k_train_data.csv", "rb"), delimiter=",",dtype=str,skiprows=0)
y_labels = y_all[0]


# initialize the y final features shape 
y_values = np.loadtxt(open("D:\\cal10k_train_data\\cal10k_train_data.csv", "rb"), delimiter=",",dtype=float,skiprows=1)
y_values = y_values[0:, 1:]     

   ## Breaking up the data into two data sets for training and crossvalidation 

In [53]:
def genCrossVal(X, y):
    X, y = shuffle(X, y, random_state=0)
    x_train = X[:5000]
    y_train = y[:5000]
    y_val = y[5000:]
    x_val = X[5000:]
    return x_train, y_train, x_val, y_val 

In [16]:
x_train, y_train, x_val, y_val = genCrossVal(dataMatrix, y_values)


(5000, 76)


# Training the logistic regression classifiers

In [96]:
# for each of the 147 different genres 
# train a classifier which is one column of the y_values matrix with the entire x_values matrix 
def trainAlgos(input_X, input_y):
    trainedAlgos = []
    X = input_X
    for i in range(input_y.shape[1]):
        y = input_y[:,i]
        logreg = LogisticRegression(C=3.5, solver='liblinear', multi_class='ovr', max_iter=1000)
        logreg.fit(X, y)
        trainedAlgos.append(logreg)
    return trainedAlgos
   


    

## Training a  support-vector machine classifier  

In [103]:
def trainSVMAlgos(input_X, input_y):
    trainedAlgos = []
    X = input_X
    for i in range(input_y.shape[1]):
        y = input_y[:,i]
        reg = SVC(kernel="rbf", gamma='scale', C=1e4, probability=True)
        reg.fit(X, y)
        trainedAlgos.append(reg)
    return trainedAlgos

##  Constructing Y_test

In [7]:
# y_test = np.zeros((2705,147), dtype=float)
# for i in range(len(trainedAlgos)):
#     y_test[:,i] = trainedAlgos[i].predict_proba(testMatrix)[:,1]

def genPredict(x_testing, trainedAlgos):
    y_test = np.zeros((x_testing.shape[0], 147), dtype=float)
    for i in range(len(trainedAlgos)):
        y_test[:,i] = trainedAlgos[i].predict_proba(x_testing)[:,1]
    return y_test
    
    
    
    

## Trying to find the best parameters for the SVM to get the best accuracy 

In [18]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.0001,0.001, 0.01, 0.1, 1, 10]
    gammas = [0.002, 0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(kernel='rbf', gamma='scale', C=0.0001), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [12]:
x_train, y_train, x_val, y_val = genCrossVal(dataMatrix, y_values)
print(svc_param_selection(x_train, y_train[:,0], KFold(n_splits=2, random_state=None, shuffle=False)))

{'C': 0.0001, 'gamma': 0.002}


In [75]:
def genSVMPredict(x_testing, trainedAlgos):
    y_test = np.zeros((x_testing.shape[0], 147), dtype=float)
    for i in range(len(trainedAlgos)):
        y_test[:,i] = trainedAlgos[i].predict_log_proba(x_testing)[:,1]
    return y_test

## Internal ROC AUC Testing for Logistic regression 

In [97]:
x_train, y_train, x_val, y_val = genCrossVal(dataMatrix, y_values)
algos = trainAlgos(x_train, y_train)
y_final = genPredict(x_val, algos)
print(roc_auc_score(y_val, y_final))

0.881790163901


## Internal ROC AUC Testing for SVM

In [104]:
x_train, y_train, x_val, y_val = genCrossVal(dataMatrix, y_values)
svmAlgos = trainSVMAlgos(x_train, y_train)
y_final = genSVMPredict(x_val, svmAlgos)
print(roc_auc_score(y_val, y_final))

0.85313127647


## Making a final submission

In [92]:
# f = open("submission1.csv", "w")
# for i in range(2705):
#     print(i, end="")
#     for j in range(147):
#         print("," +int(y_test[i][j])
              
# f.close()
algos = trainAlgos(dataMatrix, y_values)
y_test = genPredict(testMatrix, algos)
y_final = np.zeros((2705,148))
y_final[:,0] = np.arange(2705)
y_final[:,1:] = y_test




              
        

In [93]:
np.savetxt('submission16.csv', y_final, delimiter=',',fmt="%.5f")

In [148]:
x_new = [dataMatrix[2]]
y_new = logreg.predict(x_new)
print(y_new)

print(y_final[2])

[ 57.]
110.0
